In [1]:
import tkinter as tk
import customtkinter as ctk 

import torch
import numpy as np

import cv2
from PIL import Image, ImageTk
import vlc
from pydub.playback import play

import random

/opt/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [ ]:
app = tk.Tk()
app.geometry("600x600")
app.title("DD")
ctk.set_appearance_mode("dark")

vidFrame = tk.Frame(height = 480, width = 600)
vidFrame.pack()
vid = ctk.CTkLabel(vidFrame)
vid.pack()

counter = 0

counterLabel = ctk.CTkLabel(master=vidFrame, text=counter, height=40, width=120, font=("Arial", 20), text_color="white", fg_color="teal")
counterLabel.pack(pady=10)


def reset_counter():
    global counter
    counter = 0

resetButton = ctk.CTkButton(master=vidFrame, text="Reset Counter", command=reset_counter, height=40, width=120, font=("Arial", 20), text_color="white", fg_color="teal")
resetButton.pack()



model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5/runs/train/exp14/weights/last.pt', force_reload=True)
cap = cv2.VideoCapture(0)

def detect():
    global counter
    p = vlc.MediaPlayer(f"3.mp3")
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = model(frame)
    img = np.squeeze(results.render())
    
    if len(results.xywh[0]) > 0: 
        dconf = results.xywh[0][0][4]
        dclass = results.xywh[0][0][5]

        if dconf.item() > 0.30 and dclass.item() == 16.0:
            #print("yes")
            if counter > 5:
                p.play() 
            counter += 1

        if dconf.item() > 0.30 and dclass.item() == 15.0:
            p.stop()
            reset_counter()
            

    
    imgarr  = Image.fromarray(img)
    imgtk = ImageTk.PhotoImage(imgarr)
    vid.imgtk = imgtk
    vid.configure(image = imgtk)
    vid.after(10, detect)
    counterLabel.configure(text=counter)
    
detect()
app.mainloop()

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /Users/vishnu/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-4-27 Python-3.11.7 torch-2.3.0 CPU

Fusing layers... 
Model summary: 157 layers, 7055974 parameters, 0 gradients, 15.9 GFLOPs
Adding AutoShape... 
